In [1]:
import gc # garbage collector
import numpy as np # linear algebra
from collections import Counter # for counting commong words
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import matplotlib.pyplot as plt # visualization
#plt.style.use('fivethirtyeight') # use ggplot ploting style
#import seaborn as sns # visualization 
#from wordcloud import WordCloud, STOPWORDS # this module is for making wordcloud in python
import re # regular expression
import string # for finding punctuation in text
import nltk # preprocessing text
#from textblob import TextBlob
#import ploty for visualization
# import plotly.offline as py # make offline 
# py.init_notebook_mode(connected=True)
# import plotly.tools as tls
# import plotly.graph_objs as go
# from plotly.graph_objs import *
# import plotly.tools as tls
# import plotly.figure_factory as fig_fact
#plotly.tools.set_config_file(world_readable=True, sharing='public')
# remove war
import warnings
warnings.filterwarnings('ignore')
# this will allow ploting inside the notebook
#%matplotlib inline
from food_extractor.food_model import FoodModel

In [2]:
df_restaurants = pd.read_json('./new_pickles/restaurant_kmeans.json', orient = 'split', compression = 'infer')

In [21]:
df_restaurants_2 = pd.read_csv('./restaurant_kmeans_1.csv')

In [10]:
df_restaurants.to_csv('./restaurant_kmeans_1.csv', index=False)

In [25]:
df_restaurants_csv = pd.read_csv('../../restaurant_reviews.csv')

In [26]:
df_restaurants_csv.columns

Index(['review_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text'], dtype='object')

In [8]:
df_restaurants = df_restaurants.drop(df_restaurants.columns[[2, 3, 5, 10, 11,12,13,14]], axis=1)

In [4]:
df_restaurants.drop(df_restaurants[df_restaurants.is_open == 0].index, inplace=True)

In [5]:
df_restaurants['Restaurants'] = df_restaurants['categories'].str.contains('Restaurants')
df_restaurants.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Restaurants,cluster
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",True,0
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",True,4
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",True,1
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",True,3
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,1,"{'RestaurantsReservations': 'False', 'Restaura...","American (Traditional), Restaurants, Diners, B...","{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ...",True,2


In [6]:
df_restaurants = df_restaurants.loc[df_restaurants.Restaurants == True]

In [12]:
df_restaurants1 = df_restaurants

In [17]:
df_restaurants1['coordinate'] = df_restaurants1.apply(lambda row : [row['latitude'], row['longitude']], axis=1)

In [22]:
df_restaurants_2

,business_id,name,state,latitude,longitude,stars,review_count,cluster
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,PA,39.955505,-75.155564,4.0,80,0
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,TN,36.269593,-87.058943,2.0,6,4
2,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,TN,36.208102,-86.768170,1.5,10,1
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,FL,27.955269,-82.456320,4.0,10,3
4,il_Ro8jwPlHresjw9EGmBg,Denny's,IN,39.637133,-86.127217,2.5,28,2
...,...,...,...,...,...,...,...,...
34982,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,ID,43.616590,-116.202383,4.5,998,0
34983,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,PA,39.925656,-75.310344,3.0,11,2
34984,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,ID,43.615401,-116.284689,4.0,33,5
34985,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,PA,39.935982,-75.158665,4.5,35,0


In [24]:
from sklearn.cluster import KMeans


In [20]:
bu_df = pd.read_json('../../yelp_dataset/yelp_academic_dataset_business.json',lines= True)
bu_df = bu_df[bu_df['state'] == 'IL']
bu_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
38,LcAozWCMLGjwRbokaJAKMg,Edwardsville Children's Museum,722 Holyoake Rd,Edwardsville,IL,62025,38.804395,-89.949733,4.5,12,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Museums, Kids Activities, Arts & Entertainment...","{'Monday': '10:0-15:0', 'Tuesday': '9:30-14:0'..."
41,ljxNT9p0y7YMPx0fcNBGig,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,Alton,IL,62002,38.896563,-90.186203,3.0,94,1,"{'RestaurantsReservations': 'True', 'Restauran...","Restaurants, Specialty Food, Steakhouses, Food...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-21:30',..."
48,bCBPXIVfVzBZBEpFu29dcg,All In Shipping,5343 Belleville Crossing St,Belleville,IL,62226,38.517586,-90.021929,5.0,5,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services",None
86,sE6jSnvMts_MAn-b4OkMAw,K-9 Groom Room,820 Industrial Dr,Troy,IL,62294,38.716244,-89.885830,4.0,5,1,None,"Pet Groomers, Pet Sitting, Pets, Pet Services","{'Monday': '8:0-16:0', 'Tuesday': '8:0-16:0', ..."
102,EuRGgOwJ0g1vTj2R04j37Q,Crafty Crab,51 Ludwig Dr,Fairview Heights,IL,62208,38.601298,-89.989683,2.5,14,1,"{'BusinessParking': '{u'valet': False, u'garag...","Seafood, Restaurants","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'..."


In [13]:
print("In total we have", bu_df.shape[0], "restaurants.")

In total we have 2145 restaurants.


In [14]:
# To show number of open and closed businesses

op = bu_df.groupby('is_open').business_id.count()
op

is_open
0     380
1    1765
Name: business_id, dtype: int64

In [21]:
#Drop the rows where is_open=0 (those have been closed)

bu_df.drop(bu_df[bu_df.is_open == 0].index, inplace=True)
bu_df.head(200)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
38,LcAozWCMLGjwRbokaJAKMg,Edwardsville Children's Museum,722 Holyoake Rd,Edwardsville,IL,62025,38.804395,-89.949733,4.5,12,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Museums, Kids Activities, Arts & Entertainment...","{'Monday': '10:0-15:0', 'Tuesday': '9:30-14:0'..."
41,ljxNT9p0y7YMPx0fcNBGig,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,Alton,IL,62002,38.896563,-90.186203,3.0,94,1,"{'RestaurantsReservations': 'True', 'Restauran...","Restaurants, Specialty Food, Steakhouses, Food...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-21:30',..."
48,bCBPXIVfVzBZBEpFu29dcg,All In Shipping,5343 Belleville Crossing St,Belleville,IL,62226,38.517586,-90.021929,5.0,5,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services",None
86,sE6jSnvMts_MAn-b4OkMAw,K-9 Groom Room,820 Industrial Dr,Troy,IL,62294,38.716244,-89.885830,4.0,5,1,None,"Pet Groomers, Pet Sitting, Pets, Pet Services","{'Monday': '8:0-16:0', 'Tuesday': '8:0-16:0', ..."
102,EuRGgOwJ0g1vTj2R04j37Q,Crafty Crab,51 Ludwig Dr,Fairview Heights,IL,62208,38.601298,-89.989683,2.5,14,1,"{'BusinessParking': '{u'valet': False, u'garag...","Seafood, Restaurants","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16679,CYDpBk6QrNPnW9Z9hK_3qQ,Taco Bell,760 Carlyle Ave,Belleville,IL,62221,38.517048,-89.952377,2.0,33,1,"{'RestaurantsReservations': 'False', 'Restaura...","Restaurants, Breakfast & Brunch, Tacos, Mexica...","{'Monday': '8:0-23:0', 'Tuesday': '8:0-23:0', ..."
16737,c3v2SOndiJbqA_q_HYqn6Q,Drury Inn & Suites,1118 Central Park Dr,O'Fallon,IL,62269,38.574044,-89.930147,4.0,32,1,"{'WiFi': 'u'free'', 'RestaurantsPriceRange2': ...","Event Planning & Services, Hotels, Hotels & Tr...",None
16845,2Z5rz2PqzkNTkfLb44TUSA,Agostino's Restaurant and Bar,130 Ludwig Dr,Fairview Heights,IL,62208,38.600632,-89.992961,4.0,187,1,"{'Caters': 'True', 'BusinessAcceptsCreditCards...","Italian, American (Traditional), Event Plannin...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-21:0', ..."
16931,wRQ8uOse9xVRPzSvD7gFEg,Joe K's,2530 State St,Alton,IL,62002,38.920842,-90.193219,4.5,34,1,"{'NoiseLevel': 'u'average'', 'WiFi': 'u'no'', ...","Restaurants, Breakfast & Brunch, American (Tra...","{'Wednesday': '8:0-14:0', 'Thursday': '8:0-14:..."


In [22]:
bu_df['Restaurants'] = bu_df['categories'].str.contains('Restaurants')
bu_df.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Restaurants
38,LcAozWCMLGjwRbokaJAKMg,Edwardsville Children's Museum,722 Holyoake Rd,Edwardsville,IL,62025,38.804395,-89.949733,4.5,12,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Museums, Kids Activities, Arts & Entertainment...","{'Monday': '10:0-15:0', 'Tuesday': '9:30-14:0'...",False
41,ljxNT9p0y7YMPx0fcNBGig,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,Alton,IL,62002,38.896563,-90.186203,3.0,94,1,"{'RestaurantsReservations': 'True', 'Restauran...","Restaurants, Specialty Food, Steakhouses, Food...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-21:30',...",True
48,bCBPXIVfVzBZBEpFu29dcg,All In Shipping,5343 Belleville Crossing St,Belleville,IL,62226,38.517586,-90.021929,5.0,5,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services",None,False
86,sE6jSnvMts_MAn-b4OkMAw,K-9 Groom Room,820 Industrial Dr,Troy,IL,62294,38.716244,-89.885830,4.0,5,1,None,"Pet Groomers, Pet Sitting, Pets, Pet Services","{'Monday': '8:0-16:0', 'Tuesday': '8:0-16:0', ...",False
102,EuRGgOwJ0g1vTj2R04j37Q,Crafty Crab,51 Ludwig Dr,Fairview Heights,IL,62208,38.601298,-89.989683,2.5,14,1,"{'BusinessParking': '{u'valet': False, u'garag...","Seafood, Restaurants","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",True


In [23]:
df_restaurants = bu_df.loc[bu_df.Restaurants == True]
df_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Restaurants
41,ljxNT9p0y7YMPx0fcNBGig,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,Alton,IL,62002,38.896563,-90.186203,3.0,94,1,"{'RestaurantsReservations': 'True', 'Restauran...","Restaurants, Specialty Food, Steakhouses, Food...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-21:30',...",True
102,EuRGgOwJ0g1vTj2R04j37Q,Crafty Crab,51 Ludwig Dr,Fairview Heights,IL,62208,38.601298,-89.989683,2.5,14,1,"{'BusinessParking': '{u'valet': False, u'garag...","Seafood, Restaurants","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",True
678,zjQDk4tZyhEroyqtkgvx0g,The Cobblestone Eatery and Drinkery,115 W St Louis St,Lebanon,IL,62254,38.604107,-89.808533,3.5,52,1,"{'RestaurantsDelivery': 'False', 'BestNights':...","American (Traditional), Nightlife, Breakfast &...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...",True
822,gI34hM-873fwF1XQ8hOaSg,Joe’s Pizza and Pasta,"5775 Godfrey Rd, Ste A",Godfrey,IL,62035,38.949183,-90.190306,4.0,17,1,"{'HasTV': 'True', 'RestaurantsTableService': '...","Salad, Pizza, Restaurants, Italian","{'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21...",True
1145,FzelkV3vzxkRgBmVSIqALA,Edley’s,2392 S State Rt 157,Glen Carbon,IL,62034,38.773325,-89.992930,3.0,34,1,"{'OutdoorSeating': 'True', 'HasTV': 'True', 'R...","Barbeque, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",True


In [70]:
df_yelp_review=pd.read_json('../../yelp_dataset/small_review.json',orient = 'split', compression = 'infer')

In [71]:
df_yelp_review.business_id.unique()

array(['EuRGgOwJ0g1vTj2R04j37Q'], dtype=object)

In [26]:
coords = df_restaurants[['longitude','latitude']]

kmeans = KMeans(n_clusters=5, init='k-means++')
kmeans.fit(coords)


KMeans(n_clusters=5)

In [29]:
df_restaurants['cluster'] = kmeans.predict(df_restaurants[['longitude','latitude']])
df_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Restaurants,cluster
41,ljxNT9p0y7YMPx0fcNBGig,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,Alton,IL,62002,38.896563,-90.186203,3.0,94,1,"{'RestaurantsReservations': 'True', 'Restauran...","Restaurants, Specialty Food, Steakhouses, Food...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-21:30',...",True,3
102,EuRGgOwJ0g1vTj2R04j37Q,Crafty Crab,51 Ludwig Dr,Fairview Heights,IL,62208,38.601298,-89.989683,2.5,14,1,"{'BusinessParking': '{u'valet': False, u'garag...","Seafood, Restaurants","{'Monday': '12:0-22:0', 'Tuesday': '12:0-22:0'...",True,2
678,zjQDk4tZyhEroyqtkgvx0g,The Cobblestone Eatery and Drinkery,115 W St Louis St,Lebanon,IL,62254,38.604107,-89.808533,3.5,52,1,"{'RestaurantsDelivery': 'False', 'BestNights':...","American (Traditional), Nightlife, Breakfast &...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ...",True,2
822,gI34hM-873fwF1XQ8hOaSg,Joe’s Pizza and Pasta,"5775 Godfrey Rd, Ste A",Godfrey,IL,62035,38.949183,-90.190306,4.0,17,1,"{'HasTV': 'True', 'RestaurantsTableService': '...","Salad, Pizza, Restaurants, Italian","{'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21...",True,3
1145,FzelkV3vzxkRgBmVSIqALA,Edley’s,2392 S State Rt 157,Glen Carbon,IL,62034,38.773325,-89.992930,3.0,34,1,"{'OutdoorSeating': 'True', 'HasTV': 'True', 'R...","Barbeque, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",True,1


In [31]:
def recommend_restaurants(df, longitude, latitude):
    cluster = kmeans.predict(np.array([longitude,latitude]).reshape(1,-1))[0]
    print(cluster)
   
    # Get the best restaurant in this cluster
    return  df[df['cluster']==cluster].iloc[0:5][['name', 'business_id', 'latitude','longitude'	]]

In [33]:
recs_temp = recommend_restaurants(df_restaurants,38.891796, -90.186736)

2


In [63]:
recs_temp

,name,business_id,latitude,longitude
102,Crafty Crab,EuRGgOwJ0g1vTj2R04j37Q,38.601298,-89.989683
678,The Cobblestone Eatery and Drinkery,zjQDk4tZyhEroyqtkgvx0g,38.604107,-89.808533
1376,Cheddar's Scratch Kitchen,IS0B5nLJPluVT8NwGgospA,38.596518,-89.984194
2010,Wendy's,q3AbKPlzrqbUh8tz26vsdQ,38.529359,-90.035606
2250,Super Gyros,bQKqeYwaHNal7JFAXzCMFg,38.517021,-89.951385


In [59]:
recs_temp.iloc[0, :]['business_id']

'EuRGgOwJ0g1vTj2R04j37Q'

In [72]:
reviews_temp = df_yelp_review[df_yelp_review['business_id'] == recs_temp.iloc[0, :]['business_id']]

In [73]:
reviews_temp.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
472752,DrWMCBMRweRydBEk-OLKYg,h3o-SqWjDeMI2fCJI63-jg,EuRGgOwJ0g1vTj2R04j37Q,1,0,0,0,Waiter was absolutely terrible ordered our foo...,2021-11-06 02:07:15
503782,dgUJD_nBRZH8MrruBmUhLg,lt666id1nW1OzT2wZDvrQA,EuRGgOwJ0g1vTj2R04j37Q,1,0,1,1,Came on a Monday. Had to sit with everyone els...,2021-07-06 01:01:17
514047,TYqPRl8MXNjCJJVDhsGa5Q,jNxKAN9Hvwa30w9oymYt0g,EuRGgOwJ0g1vTj2R04j37Q,4,1,0,0,Drinks were good. Food was good. Appetizer was...,2021-07-01 01:29:13
524607,1IppSLNcI1bZ8wWp6mn3Ow,CSTy_x60bILiL7I2yd_DZw,EuRGgOwJ0g1vTj2R04j37Q,4,0,0,0,The food was excellent. We had a shrimp and cr...,2022-01-03 18:37:00
531166,lmaTQWT1ONixoGycJOf7XA,3WJX5-QbZyO1YLPsM5ekPw,EuRGgOwJ0g1vTj2R04j37Q,1,0,1,0,Patricia the manager/ bartender had a bad atti...,2021-12-18 02:56:39


In [74]:
def multiple_replace(string, replace_dict):
    '''
    Performs multiple string replacements at once
    '''
    pattern = re.compile('|'.join([re.escape(k) for k in sorted(replace_dict, key = len, reverse = True)]), flags = re.DOTALL)
    return pattern.sub(lambda x : replace_dict[x.group(0)], string)                  

stop_chars = []
replace_dict = {','   : '', 
                ';'   : '',
                '('   : '', 
                ')'   : '', 
                '\n' : ' ', 
                '!'   : '.',
                '?'   : '.'}

In [ ]:
reviews_temp['text'] = reviews_temp['text'].apply(lambda text: multiple_replace(text.lower(), replace_dict))


In [75]:
def get_sentiments(docs):
    '''
    Returns a Dataframe of sentiment scores with columns:
    'compound', 'pos', 'neu', 'neg'
    
    For each doc, 'pos', 'neu', 'neg' scores add to 1.
    'compound' is an overall sentiment score of the doc.
    '''
    # Instantiate SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    sentiments = []

    # Generate sentiment score for each review
    for doc in docs:
        sentiment = sia.polarity_scores(doc)
        sentiments.append(sentiment)

    return pd.DataFrame(sentiments)

In [76]:
reviews_temp_best = reviews_temp[reviews_temp['stars'] == 5]

In [92]:
reviews_temp_best_arr = reviews_temp_best['text'].tolist()


In [93]:
reviews_temp_best_arr

["Soooo good our waitress Crystal was super attentive. The shrimp were very fresh and good, crab was fantastic and can't even begin to describe the fried Oreos!",
 'My experience at the Crafty Crab was wonderful! We came in and were seated immediately, our waiter was very kind and gave us a lot of suggestions for appetizers. We went with the crab meat fries which were delicious you all should definitely give them a try! I wish I had pictures, but I was too busy enjoying the food. \n\nDefinitely check this place out if you want a great reasonably priced seafood boil!',
 'Everything is amazing ! Me and my friends went there yesterday night, the environment is amazing, huge TV screen and very clean, foods are very delicious and every time we need help always someone there help us, it is worth to the price ! I will come back next time!',
 'Perfect seafood boils! Yummy sauce, great spice level, and fresh seafood. Will be going back again!!']

In [78]:
fooditemModel = FoodModel("chambliss/distilbert-for-food-extraction")

In [95]:
foodrecs = fooditemModel.extract_foods(reviews_temp_best_arr)

In [96]:
foodrecs

[{'Product': [],
  'Ingredient': [{'text': 'shrimp',
    'span': [57, 63],
    'conf': 0.9917647242546082},
   {'text': 'crab', 'span': [90, 94], 'conf': 0.8684410452842712},
   {'text': 'fried', 'span': [146, 151], 'conf': 0.6149633526802063}]},
 {'Product': [{'text': 'Crafty Crab',
    'span': [21, 32],
    'conf': 0.8707589954137802}],
  'Ingredient': [{'text': 'crab meat fries',
    'span': [179, 194],
    'conf': 0.9971689134836197},
   {'text': 'seafood boil', 'span': [392, 404], 'conf': 0.9697426110506058}]},
 {'Product': [], 'Ingredient': []},
 {'Product': [],
  'Ingredient': [{'text': 'seafood',
    'span': [8, 15],
    'conf': 0.7575730085372925},
   {'text': 'Yummy sauce', 'span': [23, 34], 'conf': 0.9576362172762553},
   {'text': 'spice', 'span': [42, 47], 'conf': 0.9510154128074646},
   {'text': 'seafood', 'span': [65, 72], 'conf': 0.9875165820121765}]}]

In [105]:
foodlist = []

In [106]:
for i in foodrecs:
    for j in i['Ingredient']:
        if j['conf'] > 0.95:
            foodlist.append(j['text'])

In [107]:
foodlist

['shrimp',
 'crab meat fries',
 'seafood boil',
 'Yummy sauce',
 'spice',
 'seafood']